In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime

# Criando banco

In [ ]:
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_JUNK_DEMOGRAFICO" (
	"SK_DEMOGRAFICO" INTEGER NOT NULL UNIQUE,
	"DS_SEXO" VARCHAR,
	"DS_FAIXA_ETARIA" VARCHAR,
  "DT_CARGA" TEXT,
	PRIMARY KEY("SK_DEMOGRAFICO")
)''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_LOCAL_OBITO" (
	"SK_LOCAL_OBITO" INTEGER NOT NULL UNIQUE,
	"CD_LOCAL_OBITO" INTEGER,
	"DS_LOCAL_OBITO" VARCHAR,
  "DT_CARGA" TEXT,
	PRIMARY KEY("SK_LOCAL_OBITO")
)''')

cursor.execute('''
  CREATE TABLE IF NOT EXISTS "DIME_OCUPACAO" (
	"SK_OCUPACAO" INTEGER NOT NULL UNIQUE,
	"CD_CBO" INTEGER,
	"DS_OCUPACAO" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_OCUPACAO")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_DOENCA" (
	"SK_DOENCA" INTEGER NOT NULL UNIQUE,
	"CD_CID" VARCHAR,
	"DS_DOENCA" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_DOENCA")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_CIRCUNSTANCIA" (
	"SK_CIRCUNSTANCIA" INTEGER NOT NULL UNIQUE,
	"CD_CIRCUNSTANCIA" INTEGER,
	"DS_CIRCUNSTANCIA" VARCHAR,
	"DT_CARGA" TEXT,
	PRIMARY KEY("SK_CIRCUNSTANCIA")
)''')

cursor.execute('''
 CREATE TABLE IF NOT EXISTS "DIME_ESTADO" (
    "SK_ESTADO" INTEGER NOT NULL UNIQUE,
    "CD_ESTADO" INTEGER,
    "CD_UF" INTEGER,
    "DS_ESTADO" VARCHAR,
    "DS_REGIAO" VARCHAR,
		"DT_CARGA" TEXT,
    PRIMARY KEY("SK_ESTADO")
)''')


conn.commit()
conn.close()

In [ ]:
# Conexão com o banco de dados
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()



# 2. Obter a lista de tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nNúmero de linhas em cada tabela:")

# 3. Contar as linhas em cada tabela
for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    row_count = cursor.fetchone()[0]
    print(f"Tabela '{table_name}': {row_count} linhas")

# Fechar a conexão
conn.close()


Número de linhas em cada tabela:
Tabela 'DIME_JUNK_DEMOGRAFICO': 27 linhas
Tabela 'DIME_LOCAL_OBITO': 8 linhas
Tabela 'DIME_OCUPACAO': 2796 linhas
Tabela 'DIME_DOENCA': 14234 linhas
Tabela 'DIME_CIRCUNSTANCIA': 6 linhas
Tabela 'DIME_ESTADO': 28 linhas


# Inserindo Carga Manual na table DIME_JUNK_DEMOGRAFICO

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Lista de combinações (DS_SEXO, DS_FAIXA_ETARIA)
dados = [
    ("Masculino", "Recém-Nascido"),
    ("Feminino", "Recém-Nascido"),
    ("Masculino", "Bebê"),
    ("Feminino", "Bebê"),
    ("Masculino", "Criança"),
    ("Feminino", "Criança"),
    ("Masculino", "Adolescente"),
    ("Feminino", "Adolescente"),
    ("Masculino", "Jovem Adulto"),
    ("Feminino", "Jovem Adulto"),
    ("Masculino", "Adulto"),
    ("Feminino", "Adulto"),
    ("Masculino", "Idoso"),
    ("Feminino", "Idoso"),
    ("Masculino", "Mais de 100 anos"),
    ("Feminino", "Mais de 100 anos"),
    ("Não informado", "Recém-Nascido"),
    ("Não informado", "Bebê"),
    ("Não informado", "Criança"),
    ("Não informado", "Adolescente"),
    ("Não informado", "Jovem Adulto"),
    ("Não informado", "Adulto"),
    ("Não informado", "Idoso"),
    ("Não informado", "Mais de 100 anos"),
    ("Masculino", "Não informado"),
    ("Feminino", "Não informado"),
    ("Não informado", "Não informado"),
]

# Inserir sem duplicar (caso já exista)
for sexo, faixa in dados:
    cursor.execute("""
        SELECT 1 FROM DIME_JUNK_DEMOGRAFICO
        WHERE DS_SEXO = ? AND DS_FAIXA_ETARIA = ?
    """, (sexo, faixa))
    if not cursor.fetchone():
        cursor.execute("""
            INSERT INTO DIME_JUNK_DEMOGRAFICO (DS_SEXO, DS_FAIXA_ETARIA, DT_CARGA)
            VALUES (?, ?, ?)
        """, (sexo, faixa, data_carga))

# Commit e fechar
conn.commit()
conn.close()

print("✅ Dados inseridos/atualizados com sucesso!")


✅ Dados inseridos/atualizados com sucesso!


# Inserindo DIME_LOCAL_OBITO

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""

    SELECT SK_LOCAL_OBITO, CD_LOCAL_OBITO, DS_LOCAL_OBITO
    FROM DWCD_LOCAL_OBITO
    WHERE ST_SITUACAO = 'A'
""")
local_obito = cursor.fetchall()

df_local_obito = pd.DataFrame(local_obito, columns=['SK_LOCAL_OBITO', 'CD_LOCAL_OBITO', 'DS_LOCAL_OBITO'])

conn.commit()
conn.close()

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_LOCAL_OBITO
    """
)

for row in df_local_obito.itertuples(index=False):
  sk_local_obito, cd_local_obito, ds_local_obito = row
  print(sk_local_obito, cd_local_obito, ds_local_obito)
  cursor.execute(
      """
      INSERT INTO DIME_LOCAL_OBITO (SK_LOCAL_OBITO, CD_LOCAL_OBITO, DS_LOCAL_OBITO, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_local_obito, cd_local_obito, ds_local_obito, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 1 Hospital
2 2 outros estabelecimentos de saúde
3 3 Domicílio
4 4 Via pública
5 5 Outros
6 6 Aldeia indígena
7 9 Ignorado


# Inserindo na tabela DIME_OCUPACAO

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_OCUPACAO, CD_CBO, DS_OCUPACAO
    FROM DWCD_OCUPACAO
    WHERE ST_SITUACAO = 'A'
""")
ocupacao = cursor.fetchall()

df_ocupacao = pd.DataFrame(ocupacao, columns=['SK_OCUPACAO', 'CD_CBO', 'DS_OCUPACAO'])

conn.commit()
conn.close()

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_OCUPACAO
    """
)

for row in df_ocupacao.itertuples(index=False):
  sk_ocupacao, cd_cbo, ds_ocupacao = row
  print(sk_ocupacao, cd_cbo, ds_ocupacao)
  cursor.execute(
      """
      INSERT INTO DIME_OCUPACAO (SK_OCUPACAO, CD_CBO, DS_OCUPACAO, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_ocupacao, cd_cbo, ds_ocupacao, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


OperationalError: database is locked

# Inserindo na tabela DIME_DOENCA

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_DOENCA, CD_CID, DS_DOENCA
    FROM DWCD_DOENCA
    WHERE ST_SITUACAO = 'A'
""")
doenca = cursor.fetchall()

df_doenca = pd.DataFrame(doenca, columns=['SK_DOENCA', 'CD_CID', 'DS_DOENCA'])

conn.commit()
conn.close()

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_DOENCA
    """
)

for row in df_doenca.itertuples(index=False):
  sk_doenca, cd_cid, ds_doenca = row
  print(sk_doenca, cd_cid, ds_doenca)
  cursor.execute(
      """
      INSERT INTO DIME_DOENCA (SK_DOENCA, CD_CID, DS_DOENCA, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_doenca, cd_cid, ds_doenca, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 A00 Cólera
2 A000 Cólera Devida a Vibrio Cholerae 01, Biótipo Cholerae
3 A001 Cólera Devida a Vibrio Cholerae 01, Biótipo El Tor
4 A009 Cólera Não Especificada
5 A01 Febres Tifóide e Paratifóide
6 A010 Febre Tifóide
7 A011 Febre Paratifóide A
8 A012 Febre Paratifóide B
9 A013 Febre Paratifóide C
10 A014 Febre Paratifóide Não Especificada
11 A02 Outras Infecções Por Salmonella
12 A020 Enterite Por Salmonela
13 A021 Septicemia Por Salmonela
14 A022 Infecções Localizadas Por Salmonela
15 A028 Outras Infecções Especificadas Por Salmonela
16 A029 Infecção Não Especificada Por Salmonela
17 A03 Shiguelose
18 A030 Shiguelose Devida a Shigella Dysenteriae
19 A031 Shiguelose Devida a Shigella Flexneri
20 A032 Shiguelose Devida a Shigella Boydii
21 A033 Shiguelose Devida a Shigella Sonnei
22 A038 Outras Shigueloses
23 A039 Shiguelose Não Especificada
24 A04 Outras Infecções Intestinais Bacterianas
25 A040 Infecção Por Escherichia Coli Enteropatogênica
26 A041 Infecção Por Es

# Inserindo na tabela DIME_CIRCUNSTANCIA

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_CIRCUSTANCIA, CD_CIRCUNSTANCIA, DS_CIRCUNSTANCIA
    FROM DWCD_CIRCUNSTANCIA
    WHERE ST_SITUACAO = 'A'
""")
circunstancia = cursor.fetchall()

df_circunstancia = pd.DataFrame(circunstancia, columns=['SK_CIRCUSTANCIA', 'CD_CIRCUNSTANCIA', 'DS_CIRCUNSTANCIA'])

conn.commit()
conn.close()

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_CIRCUNSTANCIA
    """
)

for row in df_circunstancia.itertuples(index=False):
  sk_circunstancia, cd_circunstancia, ds_circunstancia = row
  print(sk_circunstancia, cd_circunstancia, ds_circunstancia)
  cursor.execute(
      """
      INSERT INTO DIME_CIRCUNSTANCIA (SK_CIRCUNSTANCIA, CD_CIRCUNSTANCIA, DS_CIRCUNSTANCIA, DT_CARGA)
      VALUES (?, ?, ?, ?)
      """,(sk_circunstancia, cd_circunstancia, ds_circunstancia, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado
1 1 Acidente
2 2 Suicídio
3 3 Homicídio
4 4 Outros
5 9 Ignorado


# Inserindo na tabela DIME_ESTADO

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT SK_ESTADO, CD_ESTADO, CD_UF, DS_ESTADO, DS_REGIAO
    FROM DWCD_ESTADO
    WHERE ST_SITUACAO = 'A'
""")
estado = cursor.fetchall()

df_estado = pd.DataFrame(estado, columns=['SK_ESTADO', 'CD_ESTADO', 'CD_UF', 'DS_ESTADO', 'DS_REGIAO'])

conn.commit()
conn.close()

In [ ]:
# Conexão
conn = sqlite3.connect("DM_Banco")
cursor = conn.cursor()

# Data da carga (agora)
data_carga = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

cursor.execute(
    """
    DELETE FROM DIME_ESTADO
    """
)

for row in df_estado.itertuples(index=False):
  sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao = row
  print(sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao)
  cursor.execute(
      """
      INSERT INTO DIME_ESTADO (SK_ESTADO, CD_ESTADO, CD_UF, DS_ESTADO, DS_REGIAO, DT_CARGA)
      VALUES (?, ?, ?, ?,?,?)
      """,(sk_estado, cd_estado, cd_uf, ds_estado, ds_regiao, data_carga)
  )


# Commit e fechar
conn.commit()
conn.close()


-1 -1 Não informado Não informado Não informado
1 11 RO Rondônia Norte
2 12 AC Acre Norte
3 13 AM Amazonas Norte
4 14 RR Roraima Norte
5 15 PA Pará Norte
6 16 AP Amapá Norte
7 17 TO Tocantins Norte
8 21 MA Maranhão Nordeste
9 22 PI Piauí Nordeste
10 23 CE Ceará Nordeste
11 24 RN Rio Grande do Norte Nordeste
12 25 PB Paraíba Nordeste
13 26 PE Pernambuco Nordeste
14 27 AL Alagoas Nordeste
15 28 SE Sergipe Nordeste
16 29 BA Bahia Nordeste
17 31 MG Minas Gerais Sudeste
18 32 ES Espírito Santo Sudeste
19 33 RJ Rio de Janeiro Sudeste
20 35 SP São Paulo Sudeste
21 41 PR Paraná Sul
22 42 SC Santa Catarina Sul
23 43 RS Rio Grande do Sul Sul
24 50 MS Mato Grosso do Sul Centro-Oeste
25 51 MT Mato Grosso Centro-Oeste
26 52 GO Goiás Centro-Oeste
27 53 DF Distrito Federal Centro-Oeste


# Inserindo na tabela DIME_JUNK_DETALHES

In [ ]:
conn = sqlite3.connect("DW_Banco")
cursor = conn.cursor()

cursor.execute("""
    SELECT , SK_DETALHES, SK_CIRCU
    FROM DWMV_OBITO ob
    JOIN DWCD_MUNICIPIO mu ON ob.SK_MUNICIPIO = mu.SK_MUNICIPIO
    JOIN DWCD_SEXO sx ON ob.SK_SEXO = sx.SK_SEXO
""")
obito = cursor.fetchall()

df_obito = pd.DataFrame(obito, columns=['CD_OBITO', 'DT_OBITO', 'ST_ASSISTENCIA_MEDICA', 'ST_INVESTIGADO', 'ST_EPIDEMIA','CD_OBITO', 'DT_OBITO', 'ST_ASSISTENCIA_MEDICA', 'ST_INVESTIGADO', 'ST_EPIDEMIA'])

conn.commit()
conn.close()